In [22]:
import json
import requests
from os import environ

def query(payload, api_url="https://api-inference.huggingface.co/models/gpt2"):
    API_TOKEN=environ['HF_ACCESS_TOKEN']
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    data = json.dumps(payload)
    response = requests.request("POST", api_url, headers=headers, data=data)
    ans = json.loads(response.content.decode("utf-8"))
    display([payload, api_url, ans])
    return ans

# payload = "detailed explanation of the benefits and drawbacks of using artificial intelligence in healthcare?"
payload = "Name a few capital cities of Africa?"
a0 = query(payload)

api = "https://api-inference.huggingface.co/models/bert-base-uncased"
payload = {"inputs": "The answer to the universe is [MASK]."}
query(payload,api_url=api)

api = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"
payload = {"inputs": "I really enjoyed the movie, it was great!"}
query(payload,api_url=api);


['Name a few capital cities of Africa?',
 'https://api-inference.huggingface.co/models/gpt2',
 [{'generated_text': 'Name a few capital cities of Africa?\n\nThe answer is yes.\n\nThe capital cities'}]]

[{'inputs': 'The answer to the universe is [MASK].'},
 'https://api-inference.huggingface.co/models/bert-base-uncased',
 [{'score': 0.16964051127433777,
   'token': 2053,
   'token_str': 'no',
   'sequence': 'the answer to the universe is no.'},
  {'score': 0.07344779372215271,
   'token': 2498,
   'token_str': 'nothing',
   'sequence': 'the answer to the universe is nothing.'},
  {'score': 0.05803246051073074,
   'token': 2748,
   'token_str': 'yes',
   'sequence': 'the answer to the universe is yes.'},
  {'score': 0.043957971036434174,
   'token': 4242,
   'token_str': 'unknown',
   'sequence': 'the answer to the universe is unknown.'},
  {'score': 0.04015727713704109,
   'token': 3722,
   'token_str': 'simple',
   'sequence': 'the answer to the universe is simple.'}]]

[{'inputs': 'I really enjoyed the movie, it was great!'},
 'https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english',
 [[{'label': 'POSITIVE', 'score': 0.9998735189437866},
   {'label': 'NEGATIVE', 'score': 0.0001265111204702407}]]]

''

In [26]:
from langchain.agents import load_huggingface_tool
tool = load_huggingface_tool("lysandre/hf-model-downloads")
display([{tool.name} , {tool.description}])
tool.run("text-classification")

[{'model_download_counter'},
 {'This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub. It takes the name of the category (such as text-classification, depth-estimation, etc), and returns the name of the checkpoint'}]

'facebook/bart-large-mnli'

In [23]:
from langchain import HuggingFaceHub
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = environ['HF_ACCESS_TOKEN']

repo_id = "google/flan-t5-xl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
repo_id = "google/flan-t5-small"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
# tool = load_huggingface_tool("lysandre/hf-model-downloads")

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0, "max_length": 64}, client=None)


In [28]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step.
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

for yr in range(1990, 2023,4):
    xquestion = f"Who won the football World Cup in the year {yr}? "
    print(llm_chain.run(xquestion))

The Football World Cup was held in 1990. The football world cup was held in 1990. So, the answer is 1990.
The Football World Cup was held in 1994. The World Cup was held in 1994. The answer: no.
The Football World Cup was held on 13 June 1998. The football world cup was held on 13 June 1998. The answer: yes.
The Football World Cup was held in 2002. The football world cup was held in 2002. So, the answer is 2002.
The Football World Cup was held on 28 August 2006 in the United States. The World Cup was held on 28 August 2006 in the United States. The football world cup was held on 28 August 2006 in the United States. The answer: 28 August 2006.
The Football World Cup was held on 13 August 2010. The FIFA World Cup was held on 13 August 2010. The FIFA World Cup was held on 13 August 2010. The answer: 13 August 2010.
The Football World Cup was held on 13 August 2014. The FIFA World Cup was held on 13 August 2014. The FIFA World Cup was held on 13 August 2014. The FIFA World Cup was held on 

In [32]:
# from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small", padding_size="left")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small",)

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt') #, padding='left')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    # display(bot_input_ids)

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, )

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm in!


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm a bot, and I love talking about jokes.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm a bot, and I love talking about jokes.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm a bot, and I love talking about jokes.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm a bot, and I love talking about bots.
